In [13]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact

import plotly.offline as py

import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [46]:
rd=dcm.read_file('/home/crct/CRCT/JupyterNotebook/data/image/12E/EMC_stair_e12.dcm')
#print(rd)
rdpix=rd.pixel_array.astype(np.float32)
f=float(rd.DoseGridScaling)
rdpix=rdpix*f
print(f)
ct_viewer(rdpix)
rdim=sitk.GetImageFromArray(rdpix)

origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)

spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)
print(rdpix.max())
print(rdim.GetOrigin())
print(rdim.GetSize())
print(rdim.GetSpacing())
#sitk.WriteImage(rdim, '/home/crct/CRCT/JupyterNotebook/data/image/12E/abs/emc_stair_12e.mhd')

4e-06


interactive(children=(IntSlider(value=69, description=u'myslice', max=139), Output()), _dom_classes=('widget-interact',))

4.64294
(-148.75, -101.25, -171.25)
(121, 81, 139)
(2.5, 2.5, 2.5)


In [49]:
#Copute absolute dose for Gate simulation 
gate_im=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/12E/LUNG_12E.mhd')
print(gate_im.GetOrigin())
print(gate_im.GetSize())
print(gate_im.GetSpacing())
gate_pix=sitk.GetArrayFromImage(gate_im)
gate_pix=gate_pix[:,::-1, :]

#ct_viewer(gate_pix)

GVref6=0.000108726
GVref9=0.000140945
GVref12=0.000150559

gatepix_abs=gate_pix*(1.0/GVref12)*0.01*200*1
print(gatepix_abs.max())

gatepix_abs=np.swapaxes(gatepix_abs, 0, 2)
gatepix_abs=gatepix_abs[:,:,::-1]
ct_viewer(gatepix_abs)

gate_im2=sitk.GetImageFromArray(gatepix_abs)





gate_im2.SetOrigin(gate_im.GetOrigin())
gate_im2.SetSpacing(gate_im.GetSpacing())


sitk.WriteImage(gate_im2, '/home/crct/CRCT/JupyterNotebook/data/image/12E/abs/lung_12e.mhd')


(-99.0, 50.5, -99.0)
(100, 100, 100)
(2.0, 1.0, 2.0)
1.94426


interactive(children=(IntSlider(value=50, description=u'myslice'), Output()), _dom_classes=('widget-interact',))

In [50]:
print(gatepix_abs.shape)
plot=gatepix_abs[49,20, :]
print(plot.shape)
print(plot.mean())
# Plot profil or PDD with plotly      
trace1 = go.Scatter(
    x = np.arange(0,100),
    y = plot,
    mode = 'lines', 
    name = 'test', 
)
data=[trace1]
fig= go.Figure(data=data) 
py.iplot(fig, filename='plot') 

(100, 100, 100)
(100,)
0.900529
